# Treinamento de CNN do Zero - Parte 2: Treinamento

Este notebook implementa a segunda parte do treinamento de uma Rede Neural Convolucional (CNN) do zero para classificar imagens nas mesmas categorias que usamos nos modelos YOLO.

Nesta segunda parte, vamos focar no treinamento da CNN.

## 1. Configuração do Ambiente

Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente.

In [1]:
# Verificar se o ambiente já foi configurado
import os
import sys

# Se o ambiente ainda não foi configurado, execute o setup_env.sh
if not os.path.exists('../yolov5'):
    print("Configurando o ambiente com setup_env.sh...")
    !chmod +x ../setup_env.sh
    !../setup_env.sh
else:
    print("Ambiente já configurado.")

# Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import yaml
import time
from pathlib import Path
from tqdm.notebook import tqdm

Ambiente já configurado.


## 2. Carregamento dos Dados Preparados

Vamos carregar os dados preparados na primeira parte do notebook.

In [2]:
# Verificar se os arquivos necessários existem
if not os.path.exists('../models/cnn/cnn_initial.pt'):
    print("❌ Arquivo '../models/cnn/cnn_initial.pt' não encontrado. Execute a Parte 1 primeiro.")
else:
    print("✅ Arquivo '../models/cnn/cnn_initial.pt' encontrado.")

if not os.path.exists('../models/cnn/cnn_categories.txt'):
    print("❌ Arquivo '../models/cnn/cnn_categories.txt' não encontrado. Execute a Parte 1 primeiro.")
    # Usar categorias padrão
    categories = ['apple', 'banana']
else:
    print("✅ Arquivo '../models/cnn/cnn_categories.txt' encontrado.")
    # Carregar categorias
    with open('../models/cnn/cnn_categories.txt', 'r') as f:
        categories = [line.strip() for line in f.readlines()]
    print(f"Categorias: {categories}")

# Definir diretórios de dados
train_dir = '../dataset/train/images'
val_dir = '../dataset/val/images'
test_dir = '../dataset/test/images'

# Verificar se os diretórios existem
for dir_path in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir_path):
        print(f"❌ Diretório não encontrado: {dir_path}")
    else:
        print(f"✅ Diretório encontrado: {dir_path}")
        print(f"   Número de imagens: {len([f for f in os.listdir(dir_path) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))])}")

✅ Arquivo '../models/cnn/cnn_initial.pt' encontrado.
✅ Arquivo '../models/cnn/cnn_categories.txt' encontrado.
Categorias: ['apple', 'banana']
✅ Diretório encontrado: ../dataset/train/images
   Número de imagens: 64
✅ Diretório encontrado: ../dataset/val/images
   Número de imagens: 8
✅ Diretório encontrado: ../dataset/test/images
   Número de imagens: 8


### 2.1 Recriação dos Datasets e DataLoaders

Vamos recriar os datasets e dataloaders para o treinamento.

In [3]:
# Definir transformações para as imagens
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Classe personalizada para o dataset
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, categories, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.categories = categories
        
        # Listar todas as imagens
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))]
        
        # Determinar a classe de cada imagem com base no nome do arquivo
        self.labels = []
        for img_file in self.img_files:
            # Assumindo que o nome do arquivo começa com o nome da categoria
            # Por exemplo: categoria_a_001.jpg -> categoria_a
            for i, category in enumerate(categories):
                if category.lower() in img_file.lower():
                    self.labels.append(i)
                    break
            else:
                # Se não encontrar a categoria no nome do arquivo, usar a primeira categoria
                self.labels.append(0)
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Criar datasets
train_dataset = CustomImageDataset(train_dir, categories, transform=train_transforms)
val_dataset = CustomImageDataset(val_dir, categories, transform=val_test_transforms)

# Criar dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Verificar os datasets
print(f"Tamanho do dataset de treino: {len(train_dataset)}")
print(f"Tamanho do dataset de validação: {len(val_dataset)}")

Tamanho do dataset de treino: 64
Tamanho do dataset de validação: 8


### 2.2 Recriação do Modelo

Vamos recriar o modelo CNN e carregar os pesos iniciais.

In [4]:
# Definir a arquitetura da CNN
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        
        # Camadas convolucionais
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Camadas de pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Camadas de batch normalization
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Camadas fully connected
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Bloco 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Bloco 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Bloco 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Bloco 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Criar o modelo
model = CustomCNN(num_classes=len(categories))

# Carregar os pesos iniciais
if os.path.exists('../models/cnn/cnn_initial.pt'):
    model.load_state_dict(torch.load('../models/cnn/cnn_initial.pt'))
    print("Pesos iniciais carregados com sucesso.")
else:
    print("Pesos iniciais não encontrados. Usando inicialização aleatória.")

# Verificar o número de parâmetros
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal de parâmetros: {total_params:,}")
print(f"Parâmetros treináveis: {trainable_params:,}")

Pesos iniciais carregados com sucesso.

Total de parâmetros: 26,081,026
Parâmetros treináveis: 26,081,026


## 3. Treinamento da CNN

Vamos treinar a CNN do zero.

In [5]:
# Definir o dispositivo (GPU se disponível, senão CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Mover o modelo para o dispositivo
model = model.to(device)

# Definir função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Função para treinar o modelo por uma época
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(dataloader, desc="Treinando"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zerar os gradientes
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass e otimização
        loss.backward()
        optimizer.step()
        
        # Estatísticas
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

# Função para validar o modelo
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Validando"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Estatísticas
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

Usando dispositivo: cpu


In [ ]:
# Treinar o modelo
num_epochs = 30
best_val_loss = float('inf')
best_model_path = '../models/cnn/cnn_best.pt'

# Criar diretório para salvar o modelo
os.makedirs('../models/cnn', exist_ok=True)

# Listas para armazenar métricas
train_losses = []
train_accs = []
val_losses = []
val_accs = []

# Medir o tempo de treinamento
start_time = time.time()

# Loop de treinamento
for epoch in range(num_epochs):
    print(f"Época {epoch+1}/{num_epochs}")
    
    # Treinar uma época
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validar
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Atualizar o scheduler
    scheduler.step(val_loss)
    
    # Imprimir estatísticas
    print(f"Treino - Perda: {train_loss:.4f}, Acurácia: {train_acc:.4f}")
    print(f"Validação - Perda: {val_loss:.4f}, Acurácia: {val_acc:.4f}")
    
    # Salvar o melhor modelo
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Modelo salvo em {best_model_path}")
    
    print()

# Calcular o tempo total de treinamento
training_time = time.time() - start_time
print(f"Tempo total de treinamento: {training_time:.2f} segundos")

Época 1/30


Treinando:   0%|          | 0/4 [00:00<?, ?it/s]

Validando:   0%|          | 0/1 [00:00<?, ?it/s]

Treino - Perda: 9.9185, Acurácia: 0.4531
Validação - Perda: 3.2299, Acurácia: 0.5000
Modelo salvo em ../models/cnn/cnn_best.pt

Época 2/30


Treinando:   0%|          | 0/4 [00:00<?, ?it/s]

...

Treino - Perda: 1.5569, Acurácia: 0.8906
Validação - Perda: 0.0098, Acurácia: 1.0000

Época 23/30


Treinando:   0%|          | 0/4 [00:00<?, ?it/s]

Validando:   0%|          | 0/1 [00:00<?, ?it/s]

Treino - Perda: 1.6737, Acurácia: 0.8281
Validação - Perda: 0.0134, Acurácia: 1.0000

Época 24/30


Treinando:   0%|          | 0/4 [00:00<?, ?it/s]

## 4. Visualização das Curvas de Aprendizado

Vamos visualizar as curvas de aprendizado do modelo durante o treinamento.

In [ ]:
# Plotar as curvas de aprendizado
plt.figure(figsize=(12, 5))

# Plotar a perda
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Treino')
plt.plot(val_losses, label='Validação')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.title('Curva de Perda')
plt.legend()
plt.grid(True)

# Plotar a acurácia
plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Treino')
plt.plot(val_accs, label='Validação')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.title('Curva de Acurácia')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 5. Salvando os Resultados do Treinamento

Vamos salvar as métricas de treinamento para uso na terceira parte do notebook.

In [ ]:
# Salvar as métricas de treinamento
training_metrics = {
    'train_losses': train_losses,
    'train_accs': train_accs,
    'val_losses': val_losses,
    'val_accs': val_accs,
    'training_time': training_time
}

# Salvar as métricas usando numpy
np.save('../models/cnn/cnn_training_metrics.npy', training_metrics)
print("Métricas de treinamento salvas em '../models/cnn/cnn_training_metrics.npy'")

print("\nTudo pronto para a avaliação na Parte 3!")

## 6. Próximos Passos

Na próxima parte (Parte 3), vamos:
1. Avaliar o desempenho do modelo no conjunto de teste
2. Visualizar algumas predições
3. Comparar o desempenho com os modelos YOLO